In [2]:
#!pip install nba_api

In [1]:
import nba_api

In [86]:
from nba_api.stats.endpoints import leaguegamelog
import pandas as pd
import numpy as np

gl = leaguegamelog.LeagueGameLog(season='2024-25', season_type_all_star='Regular Season')
df = gl.get_data_frames()[0]
df['HOME_OR_AWAY'] = np.where(
    df['MATCHUP'].str.contains(' vs. '), 1,
    np.where(df['MATCHUP'].str.contains(' @ '), 0, np.nan)
)
df['MATCHUP'] = df['MATCHUP'].apply(lambda x: x.split(' ')[2])
df['W/L'] = np.where(df['WL'] == 'W', 1, 0)
df = df.drop(columns=['SEASON_ID', 'TEAM_NAME', 'WL', 'VIDEO_AVAILABLE', 'PLUS_MINUS'])
df

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,HOME_OR_AWAY,W/L
0,1610612738,BOS,0022400061,2024-10-22,NYK,240,48,95,0.505,29,61,0.475,7,8,0.875,11,29,40,33,6,3,4,15,132,1.0,1
1,1610612747,LAL,0022400062,2024-10-22,MIN,240,42,95,0.442,5,30,0.167,21,25,0.840,15,31,46,22,7,8,7,22,110,1.0,1
2,1610612750,MIN,0022400062,2024-10-22,LAL,240,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,16,22,103,0.0,0
3,1610612752,NYK,0022400061,2024-10-22,BOS,240,43,78,0.551,11,30,0.367,12,16,0.750,5,29,34,20,2,3,12,12,109,0.0,0
4,1610612737,ATL,0022400064,2024-10-23,BKN,240,39,80,0.488,9,28,0.321,33,46,0.717,12,33,45,25,12,9,16,20,120,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,1610612762,UTA,0022401195,2025-04-13,MIN,240,40,89,0.449,16,42,0.381,9,11,0.818,9,31,40,24,6,5,14,15,105,0.0,0
2456,1610612763,MEM,0022401194,2025-04-13,DAL,240,52,104,0.500,15,40,0.375,13,15,0.867,19,34,53,31,13,7,12,15,132,1.0,1
2457,1610612764,WAS,0022401190,2025-04-13,MIA,240,46,90,0.511,16,37,0.432,11,17,0.647,6,37,43,33,4,3,11,14,119,0.0,1
2458,1610612765,DET,0022401192,2025-04-13,MIL,265,51,107,0.477,22,59,0.373,9,14,0.643,14,28,42,35,9,3,13,19,133,0.0,0


In [87]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

abbr_to_id = {team['abbreviation']: team['id'] for team in nba_teams}

df['OPPONENT'] = df['MATCHUP'].map(abbr_to_id).copy()
df

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,HOME_OR_AWAY,W/L,OPPONENT
0,1610612738,BOS,0022400061,2024-10-22,NYK,240,48,95,0.505,29,61,0.475,7,8,0.875,11,29,40,33,6,3,4,15,132,1.0,1,1610612752
1,1610612747,LAL,0022400062,2024-10-22,MIN,240,42,95,0.442,5,30,0.167,21,25,0.840,15,31,46,22,7,8,7,22,110,1.0,1,1610612750
2,1610612750,MIN,0022400062,2024-10-22,LAL,240,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,16,22,103,0.0,0,1610612747
3,1610612752,NYK,0022400061,2024-10-22,BOS,240,43,78,0.551,11,30,0.367,12,16,0.750,5,29,34,20,2,3,12,12,109,0.0,0,1610612738
4,1610612737,ATL,0022400064,2024-10-23,BKN,240,39,80,0.488,9,28,0.321,33,46,0.717,12,33,45,25,12,9,16,20,120,1.0,1,1610612751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,1610612762,UTA,0022401195,2025-04-13,MIN,240,40,89,0.449,16,42,0.381,9,11,0.818,9,31,40,24,6,5,14,15,105,0.0,0,1610612750
2456,1610612763,MEM,0022401194,2025-04-13,DAL,240,52,104,0.500,15,40,0.375,13,15,0.867,19,34,53,31,13,7,12,15,132,1.0,1,1610612742
2457,1610612764,WAS,0022401190,2025-04-13,MIA,240,46,90,0.511,16,37,0.432,11,17,0.647,6,37,43,33,4,3,11,14,119,0.0,1,1610612748
2458,1610612765,DET,0022401192,2025-04-13,MIL,265,51,107,0.477,22,59,0.373,9,14,0.643,14,28,42,35,9,3,13,19,133,0.0,0,1610612749


In [88]:
opp_stats = df.copy()
opp_stats = opp_stats.add_prefix('OPP_')
opp_stats = opp_stats.rename(columns={'OPP_TEAM_ID': 'OPPONENT'})
opp_stats = opp_stats.rename(columns={'OPP_GAME_ID': 'GAME_ID'})
opp_stats = opp_stats.rename(columns={'OPP_GAME_DATE': 'GAME_DATE'})
opp_stats = opp_stats.rename(columns={'OPP_MATCHUP': 'TEAM_ABBREVIATION'})
opp_stats = opp_stats.rename(columns={'OPP_MIN': 'MIN'})
opp_stats = opp_stats.rename(columns={'OPP_OPPONENT': 'TEAM_ID'})

df_full = df.merge(opp_stats, how='left', on=['OPPONENT', 'GAME_ID', 'GAME_DATE', 'MIN', 'TEAM_ID', 'TEAM_ABBREVIATION'])
df_full

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,HOME_OR_AWAY,W/L,OPPONENT,OPP_TEAM_ABBREVIATION,OPP_FGM,OPP_FGA,OPP_FG_PCT,OPP_FG3M,OPP_FG3A,OPP_FG3_PCT,OPP_FTM,OPP_FTA,OPP_FT_PCT,OPP_OREB,OPP_DREB,OPP_REB,OPP_AST,OPP_STL,OPP_BLK,OPP_TOV,OPP_PF,OPP_PTS,OPP_HOME_OR_AWAY,OPP_W/L
0,1610612738,BOS,0022400061,2024-10-22,NYK,240,48,95,0.505,29,61,0.475,7,8,0.875,11,29,40,33,6,3,4,15,132,1.0,1,1610612752,NYK,43,78,0.551,11,30,0.367,12,16,0.750,5,29,34,20,2,3,12,12,109,0.0,0
1,1610612747,LAL,0022400062,2024-10-22,MIN,240,42,95,0.442,5,30,0.167,21,25,0.840,15,31,46,22,7,8,7,22,110,1.0,1,1610612750,MIN,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,16,22,103,0.0,0
2,1610612750,MIN,0022400062,2024-10-22,LAL,240,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,16,22,103,0.0,0,1610612747,LAL,42,95,0.442,5,30,0.167,21,25,0.840,15,31,46,22,7,8,7,22,110,1.0,1
3,1610612752,NYK,0022400061,2024-10-22,BOS,240,43,78,0.551,11,30,0.367,12,16,0.750,5,29,34,20,2,3,12,12,109,0.0,0,1610612738,BOS,48,95,0.505,29,61,0.475,7,8,0.875,11,29,40,33,6,3,4,15,132,1.0,1
4,1610612737,ATL,0022400064,2024-10-23,BKN,240,39,80,0.488,9,28,0.321,33,46,0.717,12,33,45,25,12,9,16,20,120,1.0,1,1610612751,BKN,40,91,0.440,17,43,0.395,19,25,0.760,12,31,43,21,8,6,19,32,116,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,1610612762,UTA,0022401195,2025-04-13,MIN,240,40,89,0.449,16,42,0.381,9,11,0.818,9,31,40,24,6,5,14,15,105,0.0,0,1610612750,MIN,46,98,0.469,13,43,0.302,11,16,0.688,17,37,54,23,9,9,13,16,116,1.0,1
2456,1610612763,MEM,0022401194,2025-04-13,DAL,240,52,104,0.500,15,40,0.375,13,15,0.867,19,34,53,31,13,7,12,15,132,1.0,1,1610612742,DAL,38,87,0.437,5,28,0.179,16,22,0.727,10,28,38,20,6,3,18,13,97,0.0,0
2457,1610612764,WAS,0022401190,2025-04-13,MIA,240,46,90,0.511,16,37,0.432,11,17,0.647,6,37,43,33,4,3,11,14,119,0.0,1,1610612748,MIA,49,98,0.500,10,32,0.313,10,12,0.833,9,37,46,28,8,6,8,12,118,1.0,0
2458,1610612765,DET,0022401192,2025-04-13,MIL,265,51,107,0.477,22,59,0.373,9,14,0.643,14,28,42,35,9,3,13,19,133,0.0,0,1610612749,MIL,50,86,0.581,23,44,0.523,17,22,0.773,6,32,38,30,5,7,21,17,140,1.0,1


In [89]:
df_full = df_full.drop(columns=['MATCHUP', 'OPP_W/L'])
df_full

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,HOME_OR_AWAY,W/L,OPPONENT,OPP_TEAM_ABBREVIATION,OPP_FGM,OPP_FGA,OPP_FG_PCT,OPP_FG3M,OPP_FG3A,OPP_FG3_PCT,OPP_FTM,OPP_FTA,OPP_FT_PCT,OPP_OREB,OPP_DREB,OPP_REB,OPP_AST,OPP_STL,OPP_BLK,OPP_TOV,OPP_PF,OPP_PTS,OPP_HOME_OR_AWAY
0,1610612738,BOS,0022400061,2024-10-22,240,48,95,0.505,29,61,0.475,7,8,0.875,11,29,40,33,6,3,4,15,132,1.0,1,1610612752,NYK,43,78,0.551,11,30,0.367,12,16,0.750,5,29,34,20,2,3,12,12,109,0.0
1,1610612747,LAL,0022400062,2024-10-22,240,42,95,0.442,5,30,0.167,21,25,0.840,15,31,46,22,7,8,7,22,110,1.0,1,1610612750,MIN,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,16,22,103,0.0
2,1610612750,MIN,0022400062,2024-10-22,240,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,16,22,103,0.0,0,1610612747,LAL,42,95,0.442,5,30,0.167,21,25,0.840,15,31,46,22,7,8,7,22,110,1.0
3,1610612752,NYK,0022400061,2024-10-22,240,43,78,0.551,11,30,0.367,12,16,0.750,5,29,34,20,2,3,12,12,109,0.0,0,1610612738,BOS,48,95,0.505,29,61,0.475,7,8,0.875,11,29,40,33,6,3,4,15,132,1.0
4,1610612737,ATL,0022400064,2024-10-23,240,39,80,0.488,9,28,0.321,33,46,0.717,12,33,45,25,12,9,16,20,120,1.0,1,1610612751,BKN,40,91,0.440,17,43,0.395,19,25,0.760,12,31,43,21,8,6,19,32,116,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,1610612762,UTA,0022401195,2025-04-13,240,40,89,0.449,16,42,0.381,9,11,0.818,9,31,40,24,6,5,14,15,105,0.0,0,1610612750,MIN,46,98,0.469,13,43,0.302,11,16,0.688,17,37,54,23,9,9,13,16,116,1.0
2456,1610612763,MEM,0022401194,2025-04-13,240,52,104,0.500,15,40,0.375,13,15,0.867,19,34,53,31,13,7,12,15,132,1.0,1,1610612742,DAL,38,87,0.437,5,28,0.179,16,22,0.727,10,28,38,20,6,3,18,13,97,0.0
2457,1610612764,WAS,0022401190,2025-04-13,240,46,90,0.511,16,37,0.432,11,17,0.647,6,37,43,33,4,3,11,14,119,0.0,1,1610612748,MIA,49,98,0.500,10,32,0.313,10,12,0.833,9,37,46,28,8,6,8,12,118,1.0
2458,1610612765,DET,0022401192,2025-04-13,265,51,107,0.477,22,59,0.373,9,14,0.643,14,28,42,35,9,3,13,19,133,0.0,0,1610612749,MIL,50,86,0.581,23,44,0.523,17,22,0.773,6,32,38,30,5,7,21,17,140,1.0


In [91]:
X = df_full[['TEAM_ID', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'HOME_OR_AWAY', 'OPPONENT', 'OPP_FGM', 'OPP_FGA', 'OPP_FG_PCT', 'OPP_FG3M', 'OPP_FG3A', 'OPP_FG3_PCT',
        'OPP_FTM', 'OPP_FTA', 'OPP_FT_PCT', 'OPP_OREB', 'OPP_DREB', 'OPP_REB', 'OPP_AST', 'OPP_STL', 'OPP_BLK', 'OPP_TOV', 'OPP_PF', 'OPP_PTS', 'OPP_HOME_OR_AWAY']]
y_wl = df_full[['W/L']]
y_score = df_full[['PTS', 'OPP_PTS']]

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y_wl.values.ravel(), test_size=0.25, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.4764227642276423


In [93]:
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': clf.coef_[0]
}).sort_values(by='Coefficient', key=abs, ascending=False)

print(coef_df.head(10))

     Feature   Coefficient
20  OPPONENT  9.759921e-12
0    TEAM_ID  9.759919e-12
18       PTS  7.997755e-18
38   OPP_PTS -6.797852e-18
1        FGM  2.819983e-18
12       REB  2.513332e-18
21   OPP_FGM -2.365308e-18
11      DREB  2.319670e-18
13       AST  2.043246e-18
31  OPP_DREB -1.949625e-18


In [94]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE

results = []

for k in range(1, X.shape[1]+1):
    rfe = RFE(estimator=LogisticRegression(max_iter=1000), n_features_to_select=k)
    X_rfe = rfe.fit_transform(X_train, y_train)
    score = cross_val_score(LogisticRegression(max_iter=1000), X_rfe, y_train, cv=5).mean()
    results.append((k, score))

results_df = pd.DataFrame(results, columns=['Num_Features', 'CV_Accuracy'])
print(results_df.sort_values('CV_Accuracy', ascending=False).head(10))

   Num_Features  CV_Accuracy
0             1     0.507859
1             2     0.507859
2             3     0.507859
3             4     0.507859
4             5     0.507859
5             6     0.507859
6             7     0.507859
7             8     0.507859
8             9     0.507859
9            10     0.507859


## **We get subpar accuracy regardless of the number of features we add or remove**